# **INITIALIZATION**

Import libraries.

In [ ]:
# Library for reading and writing data to and from files
import os
# Library for getting dictionaries from data structures
from collections import defaultdict
# Library for reading and writing data to and from files
import pandas as pd
# Library to save the plots as html files
import plotly.io as pio
# Library which contains the graph objects that will be used to create the plots
import plotly.graph_objs as go
# Library to perform mathematical operations
import numpy as np
# Library to interpolate the data
from scipy.interpolate import griddata
# Library to do the histograms
import matplotlib.pyplot as plt

Define paths.

In [ ]:
# Dataset directory
corpus_directory='../Dataset/corpus2mw'

To iterate the reading procedure, get the names of the documents in the dataset.

In [ ]:
# Get a list with the names of the documents
texts_names = os.listdir(corpus_directory)

# **GET TOKENS**

First, define the special characters to be separated from each word.

In [ ]:
# List of characters
specialchars = [';', ':', '!', '?', '<', '>', '&', ')', '(', ']', '[', ',', '.', '"', '%', '$ ', '=', '}', '{', '-']

And define the function which separates the special characters from each word, assuming they can only be before or after each word.

In [ ]:
def token(w):
    # Init the empty list of tokens
    res = []

    # If the length is 1, add the character whatever it is
    if len(w) == 1:
        res.append(w)
    
    # Otherwise (if it's at least two characters)
    else:
        # If the first character is special, add it to the list and remove it from the word
        if w[0] in specialchars:
            res.append(w[0])
            w = w[1:]
        
        # Now, if the length became 1 because of that, for the same reason as before, add the character whatever it is
        if len(w) == 1:
            res.append(w)
        # Otherwise (if it's at least two characters), both if I had removed the first or not
        # Check whether the last character is special
        elif w[-1] in specialchars:
            res.append(w[:-1])
            res.append(w[-1])
        # or not
        else:
            res.append(w)
        
    # Return the list of tokens
    return res

For each document, for each word (in each line), if either the last or the first character are specialchars, then split in multiple tokens.

In [ ]:
corpus = []
corpus_lens = []

# For each document in the directory
for text in texts_names:

    # Init a temp empty list for the words in the current document
    words = []

    # Open the document
    with open(corpus_directory + '/' + text, 'r', errors='ignore') as file:

        # For each line
        for line in file:
            # For each word in the line
            for word in line.split():
                # Tokenize it
                aux = token(word)
                # And add each token to the list of words
                for t in aux:
                    words.append(t.lower())
    # Then append the list of tokens for the document in the corpus list
    corpus.append(words)
    corpus_lens.append(len(words))

# **STOP WORDS**

Define a function which returns the number of distinct neighbours for each word.

In [ ]:
# Function to calculate neighbour counts
def calculate_neighbour_counts():
    # Initialize a dictionary for sets
    neighbour_counts_dict = defaultdict(set)

    # For each document in the corpus
    for doc in corpus:
        # For each word in the document
        for idx in range(len(doc)):
            # If the word is not the first word, add the previous word to its set of neighbours
            if idx > 0:
                neighbour_counts_dict[doc[idx]].add(doc[idx - 1])
            # If the word is not the last word, add the next word to its set of neighbours
            if idx < len(doc) - 1:
                neighbour_counts_dict[doc[idx]].add(doc[idx + 1])

    # Convert the sets of neighbours to counts of unique neighbours
    for word, neighbours in neighbour_counts_dict.items():
        neighbour_counts_dict[word] = len(neighbours)

    # Sort the words by their counts of unique neighbours
    neighbour_counts_list = sorted(neighbour_counts_dict.items(), key=lambda item: item[1], reverse=True)

    # Remove from the neighbour_counts list the words that are special chars
    for special in specialchars:
        neighbour_counts_list = [item for item in neighbour_counts_list if item[0] != special]

    return neighbour_counts_list

Define the function to find the stop words.

In [ ]:
# Returns the extracted stop words
def get_stop_words(param=4, d=1):

    # Get the neighbours
    neighbour_counts = calculate_neighbour_counts()
    
    # Now look for the elbow point in the list
    elbow_point_index = 0
    # Iterate over the list (besides last word) to find the elbow point
    for index in range(len(neighbour_counts) - 1 - param):
        # Get the difference between the count for the current word and the next
        if abs(neighbour_counts[index][1] - neighbour_counts[index + param][1]) * d < param:
            elbow_point_index = index - 1
            break
    
    # Get the couples corresponding to all the words up to the elbow (highest num of neighbours)
    stop_word_counts = neighbour_counts[:elbow_point_index+1]
    # Get the stop words
    stop_words = [tuple[0] for tuple in stop_word_counts]
    
    # Return the stop words and the filtered expressions
    return stop_words

In [ ]:
len(get_stop_words())

Store the neighbours.

In [ ]:
nei = calculate_neighbour_counts()

And compute the stop words for various settings of parameters.

In [ ]:
# Define the ranges for param and d
param_values = range(1, 7)
d_values = [1/3, 1/2, 1, 2, 3]

# Initialize an empty list to store the results
results = []

# Iterate over all combinations of param and d
for param in param_values:
    for d in d_values:
        # Get the stop words for the current combination of param and d
        stop_words = get_stop_words(param, d)
        # Append the result as a tuple (param, d, number of stop words)
        results.append((param, d, len(stop_words)))

# Create a DataFrame from the results
df = pd.DataFrame(results, columns=['param', 'd', 'num_stop_words'])

# Pivot the DataFrame to create the desired table
pivot_df = df.pivot(index='d', columns='param', values='num_stop_words')

Get plot of the surface in a variable and store it in a HTML file.

In [ ]:
# Define x, y, z values

# Convert param_values into np.arange from range
param = np.array(param_values)
d = np.array(d_values)

stop_words = np.array([
    [26, 76, 83, 94, 94, 119],
    [40, 84, 94, 94, 121, 121],
    [61, 84, 123, 154, 154, 154],
    [61, 224, 176, 250, 176, 250],
    [61, 224, 284, 250, 284, 284]
])

# Create meshgrid for x and y
X, Y = np.meshgrid(param, d)

# Interpolate z values to complete grid
X_new = np.linspace(param.min(), param.max(), 100)
Y_new = np.linspace(d.min(), d.max(), 100)
X_new, Y_new = np.meshgrid(X_new, Y_new)
Z_new = griddata((X.flatten(), Y.flatten()), stop_words.flatten(), (X_new, Y_new), method='cubic')

# Scatter plot
scatter = go.Scatter3d(
    x=X.flatten(),
    y=Y.flatten(),
    z=stop_words.flatten(),
    mode='markers',
    marker=dict(
        size=4,
        color='rgb(23, 190, 207)',
        opacity=0.8
    )
)

# Surface plot
surface = go.Surface(
    x=X_new,
    y=Y_new,
    z=Z_new,
    colorscale='Viridis',
)

# Add point (4,1,F(4,1))
point = go.Scatter3d(
    x=[4],
    y=[1],
    z=[154],
    mode='markers',
    marker=dict(
        size=10,
        color='red',
        opacity=0.8
    )
)

# Layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='param'),
        yaxis=dict(title='d'),
        zaxis=dict(title='Number of stop words'),
        aspectratio=dict(x=1, y=1, z=0.7),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    )
)

# Create figure
fig = go.Figure(data=[scatter, surface, point], layout=layout)

# And store it in HTML
pio.write_html(fig, '../Output/3d_plot_surface.html')


Plot histogram for top 10 stop words.

In [ ]:
# Define the data
words = ['the', 'and', 'of', 'in', 'to', 'a', 'was', 'is', 'for', 'by']
frequencies = [28312, 26112, 20895, 16628, 12515, 11378, 8664, 8165, 7703, 7589]

# Create the histogram
plt.figure(figsize=(10, 6))
plt.bar(words, frequencies, color='darkgreen')
plt.xlabel('Words')
plt.ylabel('Neighbours')
plt.title('Top 10 stop words')
plt.show()


Plot the histogram for the first 500 ranked words by number of neighbours.

In [ ]:
# Get first 500 ranked words by neighbours
values = [value for key, value in nei[:500]]

# And plot the histogram
plt.figure(figsize=(10, 6))
plt.bar(range(len(values)), values, color='skyblue')
plt.xlabel('Words rank')
plt.ylabel('Neighbours')
plt.show()